In [1]:
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import trange

from function.cutpondr import CutPONDR
from function.utilities import get_uniprot_rawdata

# Param

In [2]:
#####CHANGE HERE#####
algorithm = "VSL2"  # PONDR algorithm for use (‘VLXT’, ‘XL1_XT’, ‘CAN_XT’, ‘VL3-BA’, 'VSL2')
chromedriver_path = Path("/home/wenlin/d/custom_command/chromedriver") # chrome driver's path
#####CHANGE HERE#####

In [3]:
# path for protein data downloaded from uniprot
pondr_disorder_ident_path = Path("./output/pondr_disorder_ident")

# Load uniprot data

In [4]:
human_uniprot_path = Path("./rawdata/human_uniprot.tab")
human_df = get_uniprot_rawdata(human_uniprot_path)
human_df = human_df.sort_values(by="uniprot_id", ignore_index=True)

In [5]:
human_df

,uniprot_id,gene_name,protein_name,protein_sequence,taxonomy
0,A0A024RBG1,NUDT4B,Diphosphoinositol polyphosphate phosphohydrola...,MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQW...,9606
1,A0A075B6H7,IGKV3-7,Probable non-functional immunoglobulin kappa v...,MEAPAQLLFLLLLWLPDTTREIVMTQSPPTLSLSPGERVTLSCRAS...,9606
2,A0A075B6H8,IGKV1D-42,Probable non-functional immunoglobulin kappa v...,MDMRVPAQLLGLLLLWLPGVRFDIQMTQSPSFLSASVGDRVSIICW...,9606
3,A0A075B6H9,IGLV4-69,Immunoglobulin lambda variable 4-69,MAWTPLLFLTLLLHCTGSLSQLVLTQSPSASASLGASVKLTCTLSS...,9606
4,A0A075B6I0,IGLV8-61,Immunoglobulin lambda variable 8-61,MSVPTMAWMMLLLGLLAYGSGVDSQTVVTQEPSFSVSPGGTVTLTC...,9606
...,...,...,...,...,...
20391,S4R3Y5,MTRNR2L11,Humanin-like 11 (HN11) (MT-RNR2-like protein 11),MATRGFSCLLLVISEIDLSVKRWV,9606
20392,U3KPV4,A3GALT2,"Alpha-1,3-galactosyltransferase 2 (EC 2.4.1.87...",MALKEGLRAWKRIFWRQILLTLGLLGLFLYGLPKFRHLEALIPMGV...,9606
20393,W5XKT8,SPACA6,Sperm acrosome membrane-associated protein 6 (...,MALLALASAVPSALLALAVFRVPAWACLLCFTTYSERLRICQMFVG...,9606
20394,W6CW81,PYDC5,Pyrin domain-containing protein 5 (Pyrin domai...,MESKYKEILLLTSLDNITDEELDRFKCFLPDEFNIATGKLHTLNST...,9606


# PONDR disorder identification

In [6]:
def pondr_disorder(cutpondr, input_df, algorithm):
    """
    cutpondr: CutPONDR object
    input_df: human_df for identify disorder/order sequences
    algorithm : (‘VLXT’, ‘XL1_XT’, ‘CAN_XT’, ‘VL3-BA’, 'VSL2'), default ‘VLXT’, pondr algorithm for use
    """
    output_df = pd.DataFrame(columns=["uniprot_id", "order_sequence", "disorder_sequence", "od_ident"])
    
    
    failed_list = []
    t = trange(len(input_df), desc=" ", leave=True)
    for i in t:
        uniprot_id = input_df.loc[i]["uniprot_id"]
        protein_sequence = input_df.loc[i]["protein_sequence"]

        t.set_description(uniprot_id)
        t.refresh()

        try:
            cutpondr.cut(sequence=protein_sequence, protein_name="aa", algorithm=algorithm)
            order_sequence = cutpondr.get_order_sequence()
            disorder_sequence = cutpondr.get_disorder_sequence()
            od_ident = cutpondr.get_od_ident()
            output_df = output_df.append(
                {
                    "uniprot_id": uniprot_id,
                    "order_sequence": order_sequence,
                    "disorder_sequence": disorder_sequence,
                    "od_ident": od_ident,
                },
                ignore_index=True,
            )
        except:
            t.set_description("{} failed".format(uniprot_id))
            t.refresh()
            
            print("{} failed".format(uniprot_id))
            failed_list.append(uniprot_id)
     
    
    return output_df, failed_list

In [1]:
cutpondr = CutPONDR(driver_path=chromedriver_path, show_progress_window=True)
od_human_df, failed_human_list = pondr_disorder(cutpondr, input_df=human_df, algorithm=algorithm)
cutpondr.close()

# Save output for further use

In [5]:
disorder_ident_output = pondr_disorder_ident_path / "{}_od_human_df.pkl".format(algorithm)
disorder_ident_failed_output = pondr_disorder_ident_path / "{}_failed_human_list.pkl".format(algorithm)

print(disorder_ident_output)
print(disorder_ident_failed_output)

output/pondr_disorder_ident/VSL2_od_human_df.pkl
output/pondr_disorder_ident/VSL2_failed_human_list.pkl


In [202]:
od_human_df.to_pickle(disorder_ident_output)
with open(disorder_ident_failed_output, "wb") as f:
    pickle.dump(failed_human_list, f)

In [6]:
#read
# od_human_df = pd.read_pickle(disorder_ident_output)
# with open(disorder_ident_failed_output, 'rb') as f:
#     failed_human_list = pickle.load(f)